# Compile time for GlobalQNN with and without gaussian noise

In [1]:
!pip list | grep jax
!pip list | grep horqrux

jax                      0.6.2
jax-cuda12-pjrt          0.6.2
jax-cuda12-plugin        0.6.2
jax-dft                  0.0.0
jaxlib                   0.6.2
jaxopt                   0.8.5
horqrux                  0.9.2


In [2]:
import os
from pathlib import Path
import jax
import jax.numpy as jnp
import qedft
from qedft.models.networks import GlobalQiCQNN, GlobalQiQNN, GlobalQNN
from qedft.config.config import Config
from qedft.train.od.trainer import KSDFTTrainer
from horqrux.utils.operator_utils import DiffMode
from loguru import logger

# Set the default dtype as float64
jax.config.update("jax_enable_x64", True)
# set gpu device
jax.config.update("jax_platform_name", "cuda")  # "cuda" or "cpu"

# Get project path
project_path = Path(os.path.dirname(os.path.dirname(qedft.__file__)))

# Load base configuration
config = Config(config_path=project_path / 'qedft' / 'config' / 'train_config.yaml').config

# Let's change to settings to have a global functional
config['network_type'] = 'conv_dqc'
config['use_amplitude_encoding'] = True

# Configure GlobalQNN
qicnn_config = {
    "network_type": "conv_dqc",
    "wrap_self_interaction": False,
    "wrap_with_negative_transform": False,
    "use_amplitude_encoding": False,
    "n_qubits": 6,  # Number of qubits in the quantum layer
    "n_var_layers": 8,  # Number of quantum layers
    "n_layers": 8,
    "largest_kernel_width": 3,
    "max_number_conv_layers": 1,  # Limit to 1 convolutional layer
    "list_qubits_per_layer": [],
    "force_qubits_per_layer_is_kernel_width": False,
    "normalization": 1.0,
    "last_layer_type": "dense",  # Use a linear layer at the end
    "use_bias_mlp": False,
    "last_layer_features": [1],
    "diff_mode": DiffMode.AD,
    "n_shots": 0,
    "key": jax.random.PRNGKey(0)
}

# Initialize the
#
#
#  network
network = GlobalQNN(config_dict=qicnn_config)

num_points = 513
density = jnp.linspace(0, 1, num_points)
init_fn, apply_fn = network.build_network(grids=density)

# Initialize parameters and run inference
rng_key = qicnn_config['key']
_, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
output = apply_fn(params, density)

# Print the output
print(output)

# # Initialize trainer
# trainer = KSDFTTrainer(
#     config_dict=config,
#     network=network,
#     data_path=project_path / 'data' / 'od'
# )

# # Train model
# params, loss, info = trainer.train(
#     checkpoint_save_dir=project_path / 'tests' / 'ckpts'
# )

2025-08-06 00:59:10.708 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-08-06 00:59:10.751 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6


list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-08-06 00:59:10.964 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-08-06 00:59:10.965 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:246 - Adding gaussian noise to the output of QNN with std 0.5
2025-08-06 00:59:10.966 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:281 - Adding a single dense layer at the end (outputs last layer 171)


[-10.04025798]


In [ ]:
from qedft.models.utils import count_parameters
print("Number of parameters in the Global QNN: ", count_parameters(params))

Number of parameters in the Global QNN:  315


In [4]:
print(len(params[0]))
print(len(params[1][0].flatten()))
# With 2 layers
36 + 171

144
171


207

In [ ]:
# Just quantum gates
6 * 8 * 3

144

In [6]:
import numpy as np
from horqrux.noise import DigitalNoiseInstance, DigitalNoiseType

def calculate_noise_probabilities(t1_ns, t2_ns, gate_time_ns, readout_error):
    """
    Calculate realistic noise probabilities from IBM device parameters.
    """
    # Calculate amplitude damping probability: P = 1 - exp(-t_gate/T1)
    amplitude_damping_prob = 1.0 - np.exp(-gate_time_ns / t1_ns)

    # Calculate phase damping probability: P = 1 - exp(-t_gate/T2)
    phase_damping_prob = 1.0 - np.exp(-gate_time_ns / t2_ns)

    print("\n===========================================")
    print("Summary of noise probabilities:")
    print(f"T1: {t1_ns} ns, T2: {t2_ns} ns, Gate time: {gate_time_ns} ns")
    print(f"Amplitude damping probability: {amplitude_damping_prob}")
    print(f"Phase damping probability: {phase_damping_prob}")
    print(f"Readout error: {readout_error}")

    return {
        "amplitude_damping": amplitude_damping_prob,
        "phase_damping": phase_damping_prob,
        "readout_error": readout_error,
    }

def create_realistic_noise_config(noise_scale=1.0, device_type="best"):
    """
    Creates realistic noise configuration based on IBM device parameters.
    """
    # IBM device parameters from the script
    device_params = {
        "best": {
            "t1_ns": 593.86,
            "t2_ns": 533.33,
            "gate_time_ns": 49.78,
            "readout_error": 0.212158,
        },
        "worst": {
            "t1_ns": 433.44,
            "t2_ns": 403.38,
            "gate_time_ns": 60.00,
            "readout_error": 0.504395,
        },
    }

    params = device_params[device_type]
    noise_probs = calculate_noise_probabilities(**params)

    # Scale noise probabilities
    scaled_probs = {
        key: min(prob * noise_scale, 1.0)  # Cap at 1.0
        for key, prob in noise_probs.items()
    }

    print(f"Scaled noise probabilities: {scaled_probs}")

    # Create noise instances for quantum gates
    quantum_noise = (
        DigitalNoiseInstance(
            DigitalNoiseType.AMPLITUDE_DAMPING,
            scaled_probs["amplitude_damping"]
        ),
        DigitalNoiseInstance(
            DigitalNoiseType.PHASE_DAMPING,
            scaled_probs["phase_damping"]
        ),
    )

    # Add measurement/sampling noise as Gaussian noise
    gaussian_noise_std = scaled_probs["readout_error"] * 0.1

    return {
        "noise": quantum_noise,
        "add_gaussian_noise_to_qnn_output": True,
        "gaussian_noise_std": gaussian_noise_std,
        "scaled_probs": scaled_probs
    }

# Test noise configuration with different scales
# noise_scales = [0.0, 0.1, 0.5, 1.0, 2.0, 5.0]
noise_scales = [5.0]

print("=== Testing noise configurations ===")
for scale in noise_scales:
    print(f"\\n--- Noise scale: {scale} ---")
    noise_config = create_realistic_noise_config(scale, "best")

    if scale > 0.0:
        # Create network with noise
        network_with_noise = GlobalQNN(config_dict=qicnn_config)
        init_fn_noisy, apply_fn_noisy = network_with_noise.build_network(
            grids=density,
            noise=noise_config["noise"]
        )

        # Test inference with noise
        _, params_noisy = init_fn_noisy(rng_key, input_shape=(-1, num_points, 1))
        # No jit
        output_noisy = apply_fn_noisy(params_noisy, density)

        print(f"Output with noise (scale {scale}): {output_noisy}")

        # Add Gaussian noise to output if specified
        if noise_config["add_gaussian_noise_to_qnn_output"]:
            noise_key = jax.random.PRNGKey(42)
            gaussian_noise = jax.random.normal(
                noise_key,
                shape=output_noisy.shape
            ) * noise_config["gaussian_noise_std"]
            output_with_gaussian = output_noisy + gaussian_noise
            print(f"Output with Gaussian noise: {output_with_gaussian}")
    else:
        print("No noise applied (baseline)")

print("\\n=== Noise testing completed ===")

2025-08-06 00:59:26.614 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-08-06 00:59:26.615 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-08-06 00:59:26.622 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-08-06 00:59:26.623 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:246 - Adding gaussian noise to the output of QNN with std 0.5
2025-08-06 00:59:26.623 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:281 - Adding a single dense layer at the end (outputs last layer 171)


=== Testing noise configurations ===
\n--- Noise scale: 5.0 ---

Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.40203671176044065), 'phase_damping': np.float64(0.44557253498898886), 'readout_error': 1.0}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]
Output with noise (scale 5.0): [-11.91680463]
Output with Gaussian noise: [-11.93527581]
\n=== Noise testing completed ===


In [ ]:
from qedft.models.networks import KohnShamNetwork
from qedft.models.quantum.simple_gqnn import build_simple_qnn_with_linear

class SimpleGlobalQNN(KohnShamNetwork):
    """Simple Global Quantum Neural Network using build_simple_qnn_with_linear."""

    def __init__(self, config_dict: dict | None = None):
        """Initialize from config dictionary."""
        super().__init__(config_dict)
        self.config = {
            "network_type": "mlp_ksr",
            "wrap_self_interaction": False,
            "wrap_with_negative_transform": True,
            "use_amplitude_encoding": True,
            "n_qubits": 4,
            "n_layers": 2,
            "num_batches": 171,
            "num_features": 3,
            "feature_type": "direct",
            "add_gaussian_noise_to_qnn_output": False,
            "gaussian_noise_std": 0.001,
        }
        if config_dict is not None:
            self.config.update(config_dict)

    def build_network(
        self,
        grids: jnp.ndarray,
        noise: None = None,
    ) -> tuple:
        """Build the simple global QNN using build_simple_qnn_with_linear.

        Args:
            grids: Grid points for the density functional calculations.

        Returns:
            tuple: (init_fn, apply_fn) pair of network initialization and application functions.
        """
        # Determine num_batches and num_features from grids if not set
        num_points = grids.shape[0]
        num_features = self.config.get("num_features", 3)
        num_batches = self.config.get("num_batches", num_points // num_features)

        network = build_simple_qnn_with_linear(
            n_qubits=self.config.get("n_qubits", 4),
            num_batches=num_batches,
            num_features=num_features,
            n_layers=self.config.get("n_layers", 2),
            feature_type=self.config.get("feature_type", "direct"),
            add_gaussian_noise_to_qnn_output=self.config.get("add_gaussian_noise_to_qnn_output", False),
            noise_std=self.config.get("gaussian_noise_std", 0.001),
            gate_noise=noise,
        )
        return network

: 

In [8]:
from qedft.models.quantum.simple_gqnn import build_simple_qnn_with_linear

# Noise scale
scale = 1.0

qicnn_config = {
    "network_type": "mlp_ksr",
    "use_amplitude_encoding": True,
    "n_qubits": 6,
    "n_layers": 8,
    "num_batches": 171,
    "num_features": 3,
    "feature_type": "direct",
    "add_gaussian_noise_to_qnn_output": False,
    "gaussian_noise_std": 0.001,
}

print(f"\\n--- Noise scale: {scale} ---")
noise_config = create_realistic_noise_config(scale, "best")

if scale > 0.0:
    key = jax.random.PRNGKey(0)
    init_fn_noisy, apply_fn_noisy = build_simple_qnn_with_linear(n_qubits=qicnn_config['n_qubits'], num_batches=513//3, num_features=3, n_layers=8, feature_type="direct", add_gaussian_noise_to_qnn_output=True, noise_std=0.001)

    print("=== Testing Neural XC Functional ===")
    from qedft.models.wrappers import wrap_network
    x = jnp.ones((513,))
    init_fn, apply_fn = wrap_network((init_fn_noisy, apply_fn_noisy), x, "mlp_ksr", wrap_self_interaction=False, wrap_with_negative_transform=False)
    params = init_fn(key)
    result = apply_fn(x, params)
    print(f"Result (scalar): {result}")
    print(f"Result shape: {result.shape}")

    network = SimpleGlobalQNN(config_dict=qicnn_config)

    # Initialize trainer
    trainer = KSDFTTrainer(
        config_dict=config,
        network=network,
        data_path=project_path / 'data' / 'od'
    )

    # Train model
    params, loss, info = trainer.train(
        checkpoint_save_dir=project_path / 'tests' / 'ckpts'
    )


2025-08-06 00:59:52.505 | INFO     | qedft.models.wrappers:wrap_network:205 - Global model, ensuring output is scalar (wrap_self_interaction causes global models to output (num_grids,) instead of (1,))


\n--- Noise scale: 1.0 ---

Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.08040734235208813), 'phase_damping': np.float64(0.08911450699779777), 'readout_error': 0.212158}
=== Testing Neural XC Functional ===


2025-08-06 00:59:57.434 | INFO     | qedft.train.od.trainer:__init__:58 - Initialized trainer with config: {'name': 'test', 'experiment_name': 'test', 'network_type': 'conv_dqc', 'molecule_name': 'h2', 'molecule_names': ['h2'], 'dataset1': [128, 384], 'rng': 0, 'save_plot_loss': False, 'save_every_n': 20, 'activation': 'tanh', 'n_neurons': 513, 'n_layers': 2, 'n_qubits': 9, 'n_reupload_layers': 1, 'use_rzz_parametrized_entanglers': False, 'chebychev_reuploading': False, 'add_reversed_rzz': False, 'entangling_block_type': 'alternate_linear', 'single_qubit_rotations': ['rz', 'rx', 'rz'], 'use_same_parameters': False, 'add_negative_transform': False, 'wrap_with_self_interaction_layer': False, 'wrap_with_global_functional': False, 'use_correlators_in_output': False, 'output_operators': ['Z'], 'use_bias_in_output': False, 'max_train_steps': 10000, 'factr': 1.0, 'pgtol': 1e-14, 'm': 20, 'maxfun': 10000, 'maxiter': 10000, 'num_iterations': 15, 'ks_iter_to_ignore': 10, 'discount_factor': 0.9, 

Result (scalar): 4.177025485686976
Result shape: ()


2025-08-06 00:59:58.338 | INFO     | qedft.models.wrappers:wrap_network:205 - Global model, ensuring output is scalar (wrap_self_interaction causes global models to output (num_grids,) instead of (1,))
2025-08-06 00:59:58.339 | INFO     | qedft.train.od.trainer:train:108 - Initializing fresh parameters
2025-08-06 00:59:58.342 | INFO     | qedft.train.od.trainer:train:114 - Jitting neural_xc_energy_density_fn
2025-08-06 00:59:58.546 | INFO     | qedft.train.od.trainer:train:140 - Jitting loss_fn
2025-08-06 00:59:58.548 | INFO     | qedft.train.od.trainer:train:151 - Checkpoint save directory: /home/isokolov/qex/tests/ckpts
2025-08-06 00:59:58.548 | INFO     | qedft.train.od.trainer:train:173 - Optimizing with L-BFGS-B: {'maxfun': 10000, 'factr': 1.0, 'm': 20, 'pgtol': 1e-14, 'maxiter': 10000}
2025-08-06 00:59:58.552 | INFO     | qedft.train.od.train:_kohn_sham:116 - Jitting kohn_sham_func


: 

: 

In [ ]:
# Noise scale
scale = 1.0

print(f"\\n--- Noise scale: {scale} ---")
noise_config = create_realistic_noise_config(scale, "best")

if scale > 0.0:
    # Create network with noise
    network_with_noise = GlobalQNN(config_dict=qicnn_config)
    init_fn_noisy, apply_fn_noisy = network_with_noise.build_network(
        grids=density,
        noise=noise_config["noise"]
    )

    # Test inference with noise
    _, params_noisy = init_fn_noisy(rng_key, input_shape=(-1, num_points, 1))
    output_noisy = apply_fn_noisy(params_noisy, density)

    print(f"Output with noise (scale {scale}): {output_noisy}")

    # Add Gaussian noise to output if specified
    if noise_config["add_gaussian_noise_to_qnn_output"]:
        noise_key = jax.random.PRNGKey(42)
        gaussian_noise = jax.random.normal(
            noise_key,
            shape=output_noisy.shape
        ) * noise_config["gaussian_noise_std"]
        output_with_gaussian = output_noisy + gaussian_noise
        print(f"Output with Gaussian noise: {output_with_gaussian}")

    # Initialize trainer
    trainer = KSDFTTrainer(
        config_dict=config,
        network=network,
        data_path=project_path / 'data' / 'od'
    )

    # Train model
    params, loss, info = trainer.train(
        checkpoint_save_dir=project_path / 'tests' / 'ckpts'
    )




2025-08-04 15:25:35.637 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-08-04 15:25:35.638 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-08-04 15:25:35.644 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-08-04 15:25:35.645 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:246 - Adding gaussian noise to the output of QNN with std 0.5
2025-08-04 15:25:35.645 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:281 - Adding a single dense layer at the end (outputs last layer 171)


\n--- Noise scale: 1.0 ---

Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.08040734235208813), 'phase_damping': np.float64(0.08911450699779777), 'readout_error': 0.212158}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-08-04 15:25:58.984 | INFO     | qedft.train.od.trainer:__init__:58 - Initialized trainer with config: {'name': 'test', 'experiment_name': 'test', 'network_type': 'conv_dqc', 'molecule_name': 'h2', 'molecule_names': ['h2'], 'dataset1': [128, 384], 'rng': 0, 'save_plot_loss': False, 'save_every_n': 20, 'activation': 'tanh', 'n_neurons': 513, 'n_layers': 2, 'n_qubits': 9, 'n_reupload_layers': 1, 'use_rzz_parametrized_entanglers': False, 'chebychev_reuploading': False, 'add_reversed_rzz': False, 'entangling_block_type': 'alternate_linear', 'single_qubit_rotations': ['rz', 'rx', 'rz'], 'use_same_parameters': False, 'add_negative_transform': False, 'wrap_with_self_interaction_layer': False, 'wrap_with_global_functional': False, 'use_correlators_in_output': False, 'output_operators': ['Z'], 'use_bias_in_output': False, 'max_train_steps': 10000, 'factr': 1.0, 'pgtol': 1e-14, 'm': 20, 'maxfun': 10000, 'maxiter': 10000, 'num_iterations': 15, 'ks_iter_to_ignore': 10, 'discount_factor': 0.9, 

Output with noise (scale 1.0): [-10.00914707]
Output with Gaussian noise: [-10.01306588]


2025-08-04 15:26:00.220 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-08-04 15:26:00.222 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-08-04 15:26:00.228 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-08-04 15:26:00.230 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:246 - Adding gaussian noise to the output of QNN with std 0.5
2025-08-04 15:26:00.230 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:281 - Adding a single dense layer at the end (outputs last layer 171)
2025-08-04 15:26:00.231 | INFO     | qedft.models.wrappers:wrap_network:205 - Global model, ensuring output is scalar (wrap_self_interaction causes global models to output (num_grids,) instead of (1,))
2025-08-04 1

list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-08-04 15:26:00.440 | INFO     | qedft.train.od.trainer:train:140 - Jitting loss_fn
2025-08-04 15:26:00.442 | INFO     | qedft.train.od.trainer:train:151 - Checkpoint save directory: /home/isokolov/qex/tests/ckpts
2025-08-04 15:26:00.443 | INFO     | qedft.train.od.trainer:train:173 - Optimizing with L-BFGS-B: {'maxfun': 10000, 'factr': 1.0, 'm': 20, 'pgtol': 1e-14, 'maxiter': 10000}
2025-08-04 15:26:00.449 | INFO     | qedft.train.od.train:_kohn_sham:116 - Jitting kohn_sham_func


: 

: 

# Conclusion
Still I do not understand why the compilation time is so long when running from a script.
I get like 6 min here but on the cluster like 1 hour and 12 sec per eval.